## Simulating spectral graph model frequency spectrum with `spectrome`: Usage example

![](./classes.png)

### Import modules with respective paths

In [ ]:
# this path append is for binder only
import sys
sys.path.append("../../")

#spectrome modules
from spectrome.forward import runforward
from spectrome.utils import functions, path
from spectrome.brain import Brain

#generic modules
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# Loading CONT and AD optimized params
cont_params = pd.read_csv("../results/CONT_optimizedparams.csv")
ad_params = pd.read_csv("../results/AD_optimizedparams.csv")

### Create new `Brain()` and populate it

Let us first set up a `Brain()` and its attributes: `connectome`, `ordering`, and `ntf_params`

In [ ]:
new_brain = Brain.Brain()

hcp_dir = path.get_data_path() # connectome information is in /data/ dir
new_brain.add_connectome(hcp_dir) # Use default files in /data/

Let's see what the attributes of brain are:

In [ ]:
vars(new_brain).keys()

And which have been initialized, so far

In [ ]:
vars(new_brain)

We have the connectome (`Cdk_conn`), the distance matrix (`Ddk_conn`), the permanent hcp ordering (`permHCP`), and some defaul parameters for the `network_transfer_function` (`ntf_params`).

Now, the `network_transfer_function` has 4 input parameters:
- `C`: the **reduced** connectivity matrix
- `D`: the distance matrix
- `parameters`: the 7 ntf parameters
- `w`: the frequency at which the ntf will be calculated at.
    
We have all but the first parameter since we need to convert the connectivity matrix to a **reduced** form. This is done by applying 2 functions to the `new_brain`:
- `new_brain.bi_symmetric_c()`
- `new_brain.reduce_extreme_dir()`

In [ ]:
# Some re-ordering and normalizing (reduced):
new_brain.reorder_connectome(new_brain.connectome, new_brain.distance_matrix)
new_brain.bi_symmetric_c()
new_brain.reduce_extreme_dir()

print(new_brain.reducedConnectome.shape)

For the Desikan-Killiany atlas, we have 86 brain regions.|

### Calculating network transfer function for a *range* of frequencies

Now loop over a range of frequencies of interest and calculate the network transfer function.

1. First setup such frequencies:

In [ ]:
fmin = 1 
fmax = 40
fvec = np.linspace(fmin,fmax,40)

2. Then calculate the frequency response:

In [ ]:
# Setting model parameters to be optimized for a healthy control
new_brain.ntf_params["tau_e"] = cont_params["taue"][0]/1000
new_brain.ntf_params["tau_i"] = cont_params["taui"][0]/1000
new_brain.ntf_params["alpha"] = cont_params["alpha"][0]
new_brain.ntf_params["speed"] = cont_params["speed"][0]
new_brain.ntf_params["gei"] = cont_params["gei"][0]
new_brain.ntf_params["gii"] = cont_params["gii"][0]
new_brain.ntf_params["tauC"] = cont_params["tauG"][0]/1000

# Compute for all frequencies in fvec:
model_spectrum, freq_response, eigvalues, eigvectors = runforward.run_local_coupling_forward(new_brain, new_brain.ntf_params, fvec)


The output for this example is 86 brain regions and 40 frequency bins.

### Plotting the simulated frequency spectra:

In [ ]:
for g in range(len(model_spectrum)):
    spectrum = np.abs(model_spectrum[g,:])
    plt.plot(fvec,functions.mag2db(spectrum))

plt.grid(True)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')


In [ ]:
# Setting model parameters to be optimized for a patient with AD
new_brain.ntf_params["tau_e"] = ad_params["taue"][0]/1000
new_brain.ntf_params["tau_i"] = ad_params["taui"][0]/1000
new_brain.ntf_params["alpha"] = ad_params["alpha"][0]
new_brain.ntf_params["speed"] = ad_params["speed"][0]
new_brain.ntf_params["gei"] = ad_params["gei"][0]
new_brain.ntf_params["gii"] = ad_params["gii"][0]
new_brain.ntf_params["tauC"] = ad_params["tauG"][0]/1000

# Compute for all frequencies in fvec:
model_spectrum, freq_response, eigvalues, eigvectors = runforward.run_local_coupling_forward(new_brain, new_brain.ntf_params, fvec)
for g in range(len(model_spectrum)):
    spectrum = np.abs(model_spectrum[g,:])
    plt.plot(fvec,functions.mag2db(spectrum))

plt.grid(True)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')
